In [2]:
import numpy as np
import matplotlib.pyplot as plt

# The N-Dimensional Rotational Matrix Generation Algorithm
## Source: http://article.sapub.org/10.5923.j.ajcam.20170702.04.html
### Overview

The idea behind (a simplified) version of this algorithm is that we can use a composite of multiple planar rotations in order to align some vector $p \in \mathbb{R}^{d+1}$ to an arbitrary axis $x_j \in \mathbb{R}^{d+1}$ when both points lie on a sphere $S^d$. 

In [3]:
#Given two sample vectors
x = np.array([2,26,7,2,12,65,7,32,3,5], dtype=np.float64)
z = np.array([0,0,0,0,0,0,0,0,0,1], dtype=np.float64)

A **Givens Matrix** is of form:

$$\left[
\begin{matrix}
1 & 0 & \dots & 0 & 0 & \dots & 0 & 0 \\
0 & 1 & \dots & 0 & 0 & \dots & 0 & 0 \\ 
\vdots & \vdots & \ddots & \vdots & \vdots & \ddots & \vdots & \vdots \\
0 & 0 & \dots & C_k & -S_k & \dots & 0 & 0 \\
0 & 0 & \dots & S_k & C_k & \dots & 0 & 0 \\
\vdots & \vdots & \ddots & \vdots & \vdots & \ddots & \vdots & \vdots \\
0 & 0 & \ddots & 0 & 0 & \dots & 1 & 0 \\
0 & 0 & \dots & 0 & 0 & \dots & 0 & 1 \\
\end{matrix}
\right]$$

Where $\{k,k+1\}$ are consecutive dimensions. The result is a planar rotation of the subspace spanned by $x_k,x_{k+1}$. The idea behind this is that any vector $p$ can be rotated to an axis $x_j$ via a series of rotations sequentially such that every component $x_i, i \ne j$ is set to $0$.

Each component-wise rotation is denoted by a **Givens Matrix** of form: 

$$G(k,k+1,\theta_k)$$

And the composition of all **Givens matrices**, returns the rotation matrix $R$ which transform a vector $p$ to an axis $x_j$:

$$R_{p \to x_j} = \prod_{N-1}^1 G(k,k+1,\theta_k)$$

In [5]:
def compute_planar_rotation(a):
    '''
    Define <a> to be a vector in 2 dimensional space. 
    Find the rotation matrix that aligns a to the axis z = [r,0] 
    Where r is the magnitude of <a>
    
    This method is the same as in: 
    N-dimensional Rotation Matrix Generation Algorithm
    (Zhelezov, O.I 2017)
    '''
    
    hyp = np.hypot(a[0],a[1])
    if np.greater(hyp**2,0):
        s_k = -a[1]/hyp
        c_k = a[0]/hyp
    else:
        s_k = 0
        c_k = 1
      
    return np.array([ [c_k, -s_k] , [s_k, c_k] ])

In [6]:
#Original Algorithm
N = x.shape[0] - 1
G = np.eye(N+1,N+1,dtype=np.float64)

#Initialize transformed version of x
x_bar = x
for k in np.arange(1,N+1):
    
    #Make tmp Givens: G(N-k, N-k+1,0_{N-k})
    G_k = np.eye(N+1,N+1,dtype=np.float64)
    
    #Compute subspace rotation to k+1 axis
    G_k[N-k:N-k+2,N-k:N-k+2] = compute_planar_rotation(x_bar[N-k:N-k+2])
    
    #Update x_bar
    x_bar = G_k @ x_bar
    G = G_k @ G

In [9]:
def compute_reverse_givens(a):
    '''
    Define <a> to be a vector in 2 dimensional space. 
    Find the rotation matrix that aligns a to the axis z = [0,r] 
    Where r is the magnitude of <a>
    
    This method is a slight variation (x_j = x_N instead of x_j = x_1) of
    N-dimensional Rotation Matrix Generation Algorithm
    (Zhelezov, O.I 2017)
    '''
    
    hyp = np.hypot(a[0],a[1])
    if np.greater(hyp**2,0):
        s_k = a[0]/hyp
        c_k = a[1]/hyp
    else:
        s_k = 1
        c_k = 0
    return np.array([ [c_k, -s_k] , [s_k, c_k] ])

In [10]:
#Reverse algorithm
N = x.shape[0] - 1
G = np.eye(N+1,N+1,dtype=np.float64)
x_bar = x
for k in np.arange(0,N):
    
    G_k = np.eye(N+1,N+1,dtype=np.float64)
    G_k[k:k+2,k:k+2] = compute_reverse_givens(x_bar[k:k+2])
    
    x_bar = G_k @ x_bar
    G = G_k @ G

To be moved to spherical averaging notebook

In [112]:
def inverse_exponential(q,p):
    '''
    Let <q> and <p> be two unit vectors.
    Where:
        <q> should be aligned to the x_N axis
        <p> is another vector that we want to map to the tangential hyperplane
    
    The inverse exponential map takes a point <p> and finds the angular rotation component
    between <q> and the x_i component of <p>. This returns the x_i' component of l(p) in the
    tangential hyperplane. 
    '''
    
    #Make output vector (we lose the d+1 component)
    l_p = np.zeros( p.shape[0] - 1 )
    
    #For each angular component on the sphere, find the corresponding euclidean component  
    for i in np.arange(0,l_p.shape[0]):
        
        #Compute the cosine angle in radians
        r = np.arccos(np.dot(q,p))
        
        #Project into the tangent hyperplane to get the x_i' component
        l_p[i] = p[i] * r/np.sin(r)
    
    return l_p
    

In [ ]:
def exponential_map(q,p):
    '''
    Let <q> and <p> be two vectors lying in a hyperplane. 
    Where:
        <q> is a vector in euclidean space lying on an n-sphere
        <p> is some vector lying on a hyperplane that is tangential to the n-sphere at <q>
    
    The exponential map takes a point <p> on the hyperplane and computes a point on the n-sphere
    that preserves euclidean distances in the form of angular distances between <q> and <p>. 
    '''
    
    #Make output vector (has n dimensions)
    exp_p = np.zeros(q.shape[0])
    
    #Compute the total distance of p from q in the tangential hyperplane
    r = np.linalg.norm(p)
    
    #Calculate the multiplier sin(r)/r. 
    #This if condition is mentioned in the spherical means paper pg. 11 at the bottom
    if np.greater(r,0):
        m = np.sin(r)/r
    else:
        m = 1
    
    for i in np.arange(0,p.shape[0]):
        exp_p[i] = p[i] * m
        
    #Compute the last component
    exp_p[-1] = np.cos(r)
    
    return exp_p